## Retrieve the Data

##### Load the necessary libraries, modules and data.

In [1]:
# Add Matplotlib inline magic command
%matplotlib inline

# Libraries/Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import scipy.stats as sts

In [2]:
# Files to load
city_data_to_load = "Resources/city_data.csv"
ride_data_to_load = "Resources/ride_data.csv"

In [3]:
# Read the city data file and store it in a pandas DataFrame.
city_data_df = pd.read_csv(city_data_to_load)
city_data_df.head(5)

,city,driver_count,type
0,Richardfort,38,Urban
1,Williamsstad,59,Urban
2,Port Angela,67,Urban
3,Rodneyfort,34,Urban
4,West Robert,39,Urban


In [4]:
# Read the ride data file and store it in a pandas DataFrame.
ride_data_df = pd.read_csv(ride_data_to_load)
ride_data_df.head(5)

,city,date,fare,ride_id
0,Lake Jonathanshire,2019-01-14 10:14:22,13.83,5739410935873
1,South Michelleport,2019-03-04 18:24:09,30.24,2343912425577
2,Port Samanthamouth,2019-02-24 04:29:00,33.44,2005065760003
3,Rodneyfort,2019-02-10 23:22:03,23.44,5149245426178
4,South Jack,2019-03-06 04:28:35,34.58,3908451377344


In [5]:
# Change Date from object to DateTime
ride_data_df['date'] = pd.to_datetime(ride_data_df['date'])

# Create a new column containing only date values
ride_data_df['date_only'] = ride_data_df['date'].dt.date

# Change Date only from object to DateTime
ride_data_df['date_only'] = pd.to_datetime(ride_data_df['date_only'])

# Check the formate of the 'Date' column
ride_data_df.info('date_only')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2375 entries, 0 to 2374
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   city       2375 non-null   object        
 1   date       2375 non-null   datetime64[ns]
 2   fare       2375 non-null   float64       
 3   ride_id    2375 non-null   int64         
 4   date_only  2375 non-null   datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(1), object(1)
memory usage: 92.9+ KB


In [6]:
del ride_data_df['date']


In [7]:
# Combine the data into a single dataset
pyber_data_df = pd.merge(ride_data_df, city_data_df, how="left", on=["city", "city"])

# Display the DataFrame
pyber_data_df.head()


,city,fare,ride_id,date_only,driver_count,type
0,Lake Jonathanshire,13.83,5739410935873,2019-01-14,5,Urban
1,South Michelleport,30.24,2343912425577,2019-03-04,72,Urban
2,Port Samanthamouth,33.44,2005065760003,2019-02-24,57,Urban
3,Rodneyfort,23.44,5149245426178,2019-02-10,34,Urban
4,South Jack,34.58,3908451377344,2019-03-06,46,Urban


In [8]:
# Get the number of rides for each type of city.
city_ride_count = pyber_data_df.groupby(["type"]).count()["ride_id"]
city_ride_count.head()

type
Rural        125
Suburban     625
Urban       1625
Name: ride_id, dtype: int64

In [9]:
# Get the total drivers for each type of city.
drivers_type_count = city_data_df.groupby(["type"]).sum()["driver_count"]
print(drivers_type_count)

type
Rural         78
Suburban     490
Urban       2405
Name: driver_count, dtype: int64


In [10]:
# Get the total fares for each type of city.
fare_type_total = pyber_data_df.groupby(["type"]).sum()["fare"]
print(fare_type_total)

type
Rural        4327.93
Suburban    19356.33
Urban       39854.38
Name: fare, dtype: float64


In [11]:
# Get average fare per ride for each type of city.
citytype_avg_fare = pyber_data_df.groupby(["type"]).mean()["fare"]
citytype_avg_fare.head()

type
Rural       34.623440
Suburban    30.970128
Urban       24.525772
Name: fare, dtype: float64

In [12]:
# Calculate the average fare per driver for each city type.
#driver_avg_fare = pyber_data_df.groupby(["type"]).sum()["fare"] / pyber_data_df["driver_count"].count()
driver_avg_fare = fare_type_total / drivers_type_count
driver_avg_fare.head()

type
Rural       55.486282
Suburban    39.502714
Urban       16.571468
dtype: float64

In [13]:
# Assemble summary into a DataFrame
ride_summary_df = pd.DataFrame({"Total Rides" : city_ride_count, 
                                "Total Drivers" : drivers_type_count, 
                                "Total Fares" : fare_type_total,
                                "Average Fare per Ride" : citytype_avg_fare, 
                                "Average Fare per Driver" : driver_avg_fare})

ride_summary_df

,Total Rides,Total Drivers,Total Fares,Average Fare per Ride,Average Fare per Driver
type,,,,,
Rural,125,78,4327.93,34.623440,55.486282
Suburban,625,490,19356.33,30.970128,39.502714
Urban,1625,2405,39854.38,24.525772,16.571468


In [14]:
# Formatting


ride_summary_df["Total Rides"] = ride_summary_df["Total Rides"].map("{:.0f}".format)

ride_summary_df["Total Drivers"] = ride_summary_df["Total Drivers"].map("{:.0f}".format)

ride_summary_df["Total Fares"] = ride_summary_df["Total Fares"].map("${:,.2f}".format)

ride_summary_df["Average Fare per Ride"] = ride_summary_df["Average Fare per Ride"].map("${:,.2f}".format)

ride_summary_df["Average Fare per Driver"] = ride_summary_df["Average Fare per Driver"].map("${:,.2f}".format)

# Remove the index name.
ride_summary_df.rename_axis(None, inplace=True)

ride_summary_df

,Total Rides,Total Drivers,Total Fares,Average Fare per Ride,Average Fare per Driver
Rural,125,78,"$4,327.93",$34.62,$55.49
Suburban,625,490,"$19,356.33",$30.97,$39.50
Urban,1625,2405,"$39,854.38",$24.53,$16.57


In [15]:
# Rename the columns

pyber_data_df.columns = ['City', 'Fare', 'Ride Id', 'Date', 'No. Drivers', 'City Type']
pyber_data_df.head(5)

,City,Fare,Ride Id,Date,No. Drivers,City Type
0,Lake Jonathanshire,13.83,5739410935873,2019-01-14,5,Urban
1,South Michelleport,30.24,2343912425577,2019-03-04,72,Urban
2,Port Samanthamouth,33.44,2005065760003,2019-02-24,57,Urban
3,Rodneyfort,23.44,5149245426178,2019-02-10,34,Urban
4,South Jack,34.58,3908451377344,2019-03-06,46,Urban


In [16]:
# Set the index to the Date Column
pyber_data_df.set_index('Date')
pyber_data_df.head(5)

,City,Fare,Ride Id,Date,No. Drivers,City Type
0,Lake Jonathanshire,13.83,5739410935873,2019-01-14,5,Urban
1,South Michelleport,30.24,2343912425577,2019-03-04,72,Urban
2,Port Samanthamouth,33.44,2005065760003,2019-02-24,57,Urban
3,Rodneyfort,23.44,5149245426178,2019-02-10,34,Urban
4,South Jack,34.58,3908451377344,2019-03-06,46,Urban


In [17]:
# Create a new DataFrame for fares and include only the Date, City Ytpe and Fare columns
fare_summary_df = pyber_data_df[['Date', 'City Type', 'Fare']].copy()
fare_summary_df.head(5)

,Date,City Type,Fare
0,2019-01-14,Urban,13.83
1,2019-03-04,Urban,30.24
2,2019-02-24,Urban,33.44
3,2019-02-10,Urban,23.44
4,2019-03-06,Urban,34.58


In [18]:
# Check data types
fare_summary_df.dtypes


Date         datetime64[ns]
City Type            object
Fare                float64
dtype: object

In [19]:
pyber_data_df.set_index('Date')

,City,Fare,Ride Id,No. Drivers,City Type
Date,,,,,
2019-01-14,Lake Jonathanshire,13.83,5739410935873,5,Urban
2019-03-04,South Michelleport,30.24,2343912425577,72,Urban
2019-02-24,Port Samanthamouth,33.44,2005065760003,57,Urban
2019-02-10,Rodneyfort,23.44,5149245426178,34,Urban
2019-03-06,South Jack,34.58,3908451377344,46,Urban
...,...,...,...,...,...
2019-04-29,Michaelberg,13.38,8550365057598,6,Rural
2019-01-30,Lake Latoyabury,20.76,9018727594352,2,Rural
2019-02-10,North Jaime,11.11,2781339863778,1,Rural


In [20]:
# Get the total fares for each type of city.
fare_citytype_total = pyber_data_df.groupby(["City Type", "Date"]).sum()["Fare"]
fare_citytype_total.head(25)

City Type  Date      
Rural      2019-01-01     43.69
           2019-01-02     52.12
           2019-01-03     19.90
           2019-01-04     24.88
           2019-01-06     47.33
           2019-01-08     19.39
           2019-01-09     31.84
           2019-01-11     16.42
           2019-01-14     72.15
           2019-01-15     30.26
           2019-01-18     58.69
           2019-01-19    144.90
           2019-01-22     23.61
           2019-01-23     40.24
           2019-01-24     34.74
           2019-01-27     81.10
           2019-01-28     31.25
           2019-01-30    109.20
           2019-01-31     58.29
           2019-02-01     17.05
           2019-02-02     65.49
           2019-02-03     51.80
           2019-02-04     59.67
           2019-02-05     45.02
           2019-02-10     11.11
Name: Fare, dtype: float64

In [21]:
pyber_data_df.set_index('Date')

,City,Fare,Ride Id,No. Drivers,City Type
Date,,,,,
2019-01-14,Lake Jonathanshire,13.83,5739410935873,5,Urban
2019-03-04,South Michelleport,30.24,2343912425577,72,Urban
2019-02-24,Port Samanthamouth,33.44,2005065760003,57,Urban
2019-02-10,Rodneyfort,23.44,5149245426178,34,Urban
2019-03-06,South Jack,34.58,3908451377344,46,Urban
...,...,...,...,...,...
2019-04-29,Michaelberg,13.38,8550365057598,6,Rural
2019-01-30,Lake Latoyabury,20.76,9018727594352,2,Rural
2019-02-10,North Jaime,11.11,2781339863778,1,Rural


In [22]:
# Create a pivot table Data Frame with the Date as the index
fare_table = pd.pivot_table(pyber_data_df, values = 'Fare', index = ['Date'], columns = ['City Type'], aggfunc = np.sum)
fare_table

City Type,Rural,Suburban,Urban
Date,,,
2019-01-01,43.69,128.52,127.78
2019-01-02,52.12,274.92,305.99
2019-01-03,19.90,133.48,260.48
2019-01-04,24.88,20.69,258.14
2019-01-05,NaN,101.36,506.89
...,...,...,...
2019-05-04,NaN,91.71,323.57
2019-05-05,26.53,154.78,322.10
2019-05-06,111.88,34.12,181.20


In [31]:
# Save the updated dataframe
export_csv = fare_table.to_csv(r'Resources/Faresbydate.csv', index = ['Date'], header=True)

In [33]:
fare_data_to_load = "Resources/faresbydate.csv"

In [34]:
fare2_data_df = pd.read_csv(fare_data_to_load)
fare2_data_df.head(5)

,Date,Rural,Suburban,Urban,Month
0,1/1/19,43.69,128.52,127.78,Jan
1,1/2/19,52.12,274.92,305.99,Jan
2,1/3/19,19.90,133.48,260.48,Jan
3,1/4/19,24.88,20.69,258.14,Jan
4,1/5/19,NaN,101.36,506.89,Jan


In [29]:
fare2_data_df = fare2_data_df.pivot(index='index', columns='Month', values)

KeyError: 'Date'